# Welcome to the HRRR data notebook!
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Extract data from the most recently updated HRRR run to produce a 12-hour forecast for a latitude/longitude point. 
#### **Outcome:** A 12-hour forecast from HRRR's latest model run in csv format, and a graphical depiction of key forecast variables.          

This notebook is an introduction to accessing HRRR data from the AWS Cloud. The first block of code, when run, produces a csv file populated with forecasted surface variables from the latest HRRR run, which cover an extent of 12 hours from the current time step. If desired, this code block can be run automatically, at a certain time or on computer start-up, with the assistance of other programs. The remaining blocks of code are meant to provide a rudimentary visualization of the variables forecasted for the next 12 hours. 

## Importing

In [ ]:
import s3fs
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import numcodecs as ncd
import metpy
import metpy.units as mu
import metpy.calc as mc
import pandas as pd 
import xarray as xr
import cartopy.crs as ccrs

## Definitions and Data Organization

In [ ]:
def get_nearest_point(projection, chunk_index, longitude, latitude):
    x, y = projection.transform_point(longitude, latitude, ccrs.PlateCarree())
    return chunk_index.sel(x=x, y=y, method="nearest")

def retrieve_data(s3_url):
    with fs.open(s3_url, 'rb') as compressed_data: # using s3fs
        buffer = ncd.blosc.decompress(compressed_data.read())

        dtype = "<f2"
        if "surface/PRES" in s3_url: # surface/PRES is the only variable with a larger data type
            dtype = "<f4"

        chunk = np.frombuffer(buffer, dtype=dtype)
        
        entry_size = 150*150
        num_entries = len(chunk)//entry_size

        if num_entries == 1: # analysis file is 2d
            data_array = np.reshape(chunk, (150, 150))
        else:
            data_array = np.reshape(chunk, (num_entries, 150, 150))

    return data_array

# Lists
forecast_hour = []
date_list = []
temp_list = []
pres_list = []
snow_list = []
gust_list = []
accum_snow = []
rh_list = []
ugrd_list = []
vgrd_list = []
wind_list = []
carinal_wind_dir = []

surface_list = ["SNOD", "GUST", "ASNOW_acc_fcst", "PRES","TMP"]
twom_list = ["RH"]
tenm_list = ["UGRD", "VGRD", "WIND_1hr_max_fcst"]

# Location
point_lat = 39.58148838130895
point_lon = -105.94259648925797

# Find chunk that contains coordinates
fs = s3fs.S3FileSystem(anon=True)

chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,
                                                     semiminor_axis=6371229))
nearest_point = get_nearest_point(projection, chunk_index, point_lon, point_lat)
fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"

# Date range:
now = datetime.now() - timedelta(hours=3)
day = now.strftime("%Y%m%d")
hr = now.strftime("%H")

for var in surface_list:
    level = 'surface'
    data_url = f'hrrrzarr/sfc/{day}/{day}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

    data = retrieve_data(data_url + fcst_chunk_id)
    gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
        
    for n in range(3, 15):
        if var == "PRES":
            pres_list.append(gridpoint_forecast[n])
            date_list.append(day)
            forecast_hour.append(n + int(hr))
        if var == "TMP":
            temp_list.append(gridpoint_forecast[n])
        elif var == "SNOD":
            snow_list.append(gridpoint_forecast[n])
        elif var == "GUST":
            gust_list.append(gridpoint_forecast[n])
        elif var == "ASNOW_acc_fcst":
            accum_snow.append(gridpoint_forecast[n])
            
for var in twom_list:
    level = '2m_above_ground'
    data_url = f'hrrrzarr/sfc/{day}/{day}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

    data = retrieve_data(data_url + fcst_chunk_id)
    gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]

    for n in range(3, 15):
        if var == "RH":
            rh_list.append(gridpoint_forecast[n])
            
for var in tenm_list:
    level = '10m_above_ground'
    data_url = f'hrrrzarr/sfc/{day}/{day}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

    data = retrieve_data(data_url + fcst_chunk_id)
    gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]

    for n in range(3, 15):
        if var == "UGRD":
            ugrd_list.append(gridpoint_forecast[n])
        elif var == "VGRD":
            vgrd_list.append(gridpoint_forecast[n])
        elif var == "WIND_1hr_max_fcst":
            wind_list.append(gridpoint_forecast[n])

# Calculate wind direction
for i in range(0, len(ugrd_list)):
    direction = metpy.calc.wind_direction(ugrd_list[i]*mu.units.metre/mu.units.second, vgrd_list[i]*mu.units.metre/mu.units.second)
    cardinal_direction = direction.magnitude
    carinal_wind_dir.append(mc.angle_to_direction(int(cardinal_direction)))

# Put output into a dataframe
output = pd.DataFrame(
    {'Date': date_list,
     'Hour_UTC': forecast_hour,
     'Pres': pres_list,
     'Temp': temp_list,
     'Snow_Depth': snow_list,
     'Hourly_Snow': accum_snow,
     'RH': rh_list,
     'Gust': gust_list,
     'Wind_Dir': carinal_wind_dir,
     'Wind_Speed': wind_list,
    })

# Unit conversions 
output.Date = pd.to_datetime(output.Date, format='%Y%m%d')
output.loc[output["Hour_UTC"] >= 24, "Date"] = output["Date"] + timedelta(days=1)
output.loc[output["Hour_UTC"] >= 24, "Hour_UTC"] = output["Hour_UTC"] - 24
output.Date = output.Date.dt.strftime('%m/%d/%y')
output.Pres = output.Pres / 3386 # inHg
output.Temp = (output.Temp - 273.15) * (9/5) + 32 # F
output.Snow_Depth = output.Snow_Depth * 39.37 # in
output.Hourly_Snow = output.Hourly_Snow * 39.37 # in
output.Gust = output.Gust * 2.24 # mph

print(output)

# Put output into CSV file
output.to_csv('weather.csv', index=False)

## Visualize the 12-hour forecast.

In [ ]:
import matplotlib.pyplot as plt

print(f"Current date/time: {datetime.now()} UTC")
print(f"HRRR data from: {hr} UTC")

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, sharex=True)
output.Temp.plot(ax=ax1, color = "red", label = "Temperature (F)")
output.Temp.plot(ax=ax1, color = "red", label = "Temperature (F)")
output.RH.plot(ax=ax2, color = "black", label = "Relative Humidity (%)")
output.Gust.plot(ax=ax3, color = "orange", label = "Wind Gust (mph)")
output.Hourly_Snow.plot.bar(ax=ax4, color = "darkblue", label = "Total Snowfall (in)")
output.Snow_Depth.plot.bar(ax=ax5, color = "lightblue", label = "Snow Depth (in)")
ax1.legend()
ax1.grid(alpha=0.4)
ax1.hlines(y=32, xmin=0, xmax=12, linewidth=.5, linestyles='--', color='blue')
ax2.legend()
ax2.grid(alpha=0.4)
ax3.legend()
ax3.grid(alpha=0.4)
ax4.legend()
ax4.grid(alpha=0.4)
ax5.legend()
ax5.grid(alpha=0.4)
ax1.set_xticklabels(output.Hour_UTC)
plt.xlabel("Time (UTC)")

**HRRR Product Documentation:**
- https://rapidrefresh.noaa.gov/hrrr/ (Product page)
- https://journals.ametsoc.org/view/journals/wefo/37/8/WAF-D-21-0151.1.xml (Academic paper)
- https://rapidrefresh.noaa.gov/RAPv4HRRRv3-OpsTrim.jpg (Domains map)
- https://rapidrefresh.noaa.gov/Diag-vars-NOAA-TechMemo.pdf (Variable description)

**Coding Sources:**
- https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/ex_python_plot_zarr.html      
- https://github.com/ldicarlo1/weather_forecast_bias_correction/blob/main/Bias%20Correction%20of%20Weather%20Forecasts%20Using%20Machine%20Learning.pdf
- https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html     
- https://registry.opendata.aws/noaa-hrrr-pds/

**CSP Access:**
- AWS: https://registry.opendata.aws/noaa-hrrr-pds/     
- Azure: https://microsoft.github.io/AIforEarthDataSets/data/noaa-hrrr.html
- Google: https://console.cloud.google.com/marketplace/details/noaa-public/hrrr?supportedpurview=project

The unique component of this Jupyter notebook is that you are not requried to download any datasets -- all data will be pulled directly from the cloud. You can learn more about NOAA's efforts to move more data to the cloud at this site: https://www.noaa.gov/nodd/datasets. As we continute to make more data widely accessible on the cloud, we'll also create more Jupyter notebooks like this one, so anyone can visualize weather and climate data without any cost or restriction. 